In [1]:
import os, shutil
import subprocess as subp

In [2]:
# number of simulations per epoch
N = 16
grompp = ["gmx","grompp"]

In [5]:
def init_rep(i):
    """ Initializes rep i
    """
    os.makedirs("epoch01/rep%02d"%i, exist_ok=True)
    shutil.copyfile("initial/start%02d.gro"%i, "epoch01/rep%02d/start.gro"%i)
    print("Copied initial/start%02d.gro, starting to grompp..."%i)
    
    prevdir = os.getcwd()
    os.chdir("epoch01/rep%02d"%i)
    
    cmd = grompp + ["-c", "start.gro", "-f", "../../mdrun.mdp", "-n", "../../index_grompp.ndx", \
                    "-p", "../../topol.top", "-o", "mdrun.tpr", "-maxwarn", "1"]

    with open("output_grompp.txt", "w") as fout:
        compProc = subp.run(cmd, stdout=fout, stderr=subp.STDOUT)
    
    os.chdir(prevdir)
    print("Process returned %d"%compProc.returncode)

In [6]:
# Initial structures and first epoch
os.makedirs("epoch01", exist_ok=True)

for i in range(1,N+1):
    init_rep(i)

Copied initial/start01.gro, starting to grompp...
Process returned 0
Copied initial/start02.gro, starting to grompp...
Process returned 0
Copied initial/start03.gro, starting to grompp...
Process returned 0
Copied initial/start04.gro, starting to grompp...
Process returned 0
Copied initial/start05.gro, starting to grompp...
Process returned 0
Copied initial/start06.gro, starting to grompp...
Process returned 0
Copied initial/start07.gro, starting to grompp...
Process returned 0
Copied initial/start08.gro, starting to grompp...
Process returned 0
Copied initial/start09.gro, starting to grompp...
Process returned 0
Copied initial/start10.gro, starting to grompp...
Process returned 0
Copied initial/start11.gro, starting to grompp...
Process returned 0
Copied initial/start12.gro, starting to grompp...
Process returned 0
Copied initial/start13.gro, starting to grompp...
Process returned 0
Copied initial/start14.gro, starting to grompp...
Process returned 0
Copied initial/start15.gro, starti